In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import datasets
from sklearn.metrics import accuracy_score
from random import choices , randint, randrange , random,uniform
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,plot_tree
from operator import itemgetter


# Define the genetic algorithm parameters
population_size = [5,7]
generation_size = [10,20]
mutation_rate = 0.02
# best_feature_score_list=[]

#this function will generate a list of 0 and 1,of lenght equal to the total number of chromosomes in dataset
#basically it will create our chromosome..
def generate_chromosome(x_df):
    return [randint(0, 1) for _ in range(x_df.shape[1])]


#this population will create our population
#it will call generate_chromosome() function number of times = population_size and and append all chromosome into list return a population

def init_population(population_size,x_df):
  population = []
  for i in range(population_size):
    chromosome=generate_chromosome(x_df)
    population.append(chromosome)
  return population



#convert actual chromosome into seleccted features dataframe
#this function will iterate over our chromosome and the indices it will find 1, it will add the feature of that indices to our
#new DataFrame

def select_feature(chromosome,x_df):
  new_df=pd.DataFrame()
  for i in range(len(chromosome)):
    if(chromosome[i]==1):
      column_name=x_df.columns[i]       # ith column name extract from the dataframe x_df
      data=x_df[column_name]            #with the help of column_name extact column data
      new_df[column_name]=data          #data will insert into new dataframe
  # print(new_df.shape)
  return new_df


def fitness_function(chromosome,x_df,y_df):
  ct=0
  for i in chromosome:        #if chromosome generate[0,0,0,0,......] No feature selected thorw error
    if i==0:
      ct+=1

  if ct== len(chromosome):          #if count is equal to length of chromosome flip 0th bit
    chromosome[0]=1

  select_feature_df=pd.DataFrame()
  select_feature_df=select_feature(chromosome,x_df)      #returns selected features DataFrame

  xtrain , xtest, ytrain, ytest = train_test_split(select_feature_df, y_df)            #split selected feature dataframe traing and testing

  clf=DecisionTreeClassifier()        #create DecisionTreeClassifier

  clf.fit(xtrain,ytrain)      #train classifier
  y_predict=clf.predict(xtest)
  score=accuracy_score(ytest,y_predict)    #calculate accuracy score between test target and predicted target
  return score



#this function will divide our population into two parts fit and unfit, it will send the fit ones to the production
# and it wil discrad the unfit one---- survival of the fittest

def selection(population,x_df,y_df):        #initial population selection
  fitness_score=[]
  for chromosome in population:       #for each chromosome in the populaton we will calculate it's fitness score
    score=fitness_function(chromosome,x_df,y_df)        #fitness function calculate chromosome(list) fitness score
    fitness_score.append(score)         #and we will append it to the list..
  indx=np.argsort(fitness_score)[::-1]  #sort fitness score in desending order and the get he indexes of them ..
                                        #through these index we select those population which is more fit

  selected_population=[]
  for i in indx[:int(len(population)/2)]:      # Itreate loop to half of the population and discard those population who is less fit
    selected_population.append(population[i])   #now based on the above sorted indexes of fitness_score, select popolation which generate heigher fitness score

  # best_feature_score_list.append([k,selected_population[0],fitness_function(selected_population[0],x_df,y_df)]) #here we are addign best of each generation into our list for further knowledge
  return selected_population


#this function will use the roullete wheel method to pick the parents further for the crossover..

def select_parent(population,x_df,y_df):
  fit_score=[]
  for chromosome in population:
    score=fitness_function(chromosome,x_df,y_df)  #calculate all fitness score of chromosome in population
    fit_score.append(score)             #append it fit_score list

#roullete wheel method
  total_score=sum(fit_score)           #calculate sum of all fit score for probability
  selection_probability=[]
  for chromosome_fitness in fit_score:   #itrate fit_score for calculating probability
    selection_probability.append(chromosome_fitness/total_score) #calculate probabiltiy of all chromosome based on thier fitness score/total score and apend to select probability
  parent=choices(population,weights=selection_probability,k=2)       #select 2 parents(k=2) from the population by passing probability
  return parent      # we will select two parents and return them to the calling function.


#crossover , single point crossover..
def crossover(parent1,parent2):
  cross_over_point=randint(1,len(parent1)-1)   #generate a random index.. from 1 to length-1
  offspring1=parent1[:cross_over_point]+parent2[cross_over_point:]  #and do the crossover..  cut and append
  offspring2=parent2[:cross_over_point]+parent1[cross_over_point:]  #smilarliy generate offspring2 based on cross_over_point slice it
  return offspring1,offspring2  #return two produced offsprings


#mutation
def mutation(child1, mut_prob):

  index=randint(0,len(child1)-1)    #generate random index for mutate ith position of offspring

  # child1 = toList(child1)
  if(mut_prob > random()):     #select a random number from 0-1 and if it is smaller than mutation probabity than flip
    if(child1[index]==1):            #a bit at randomly generated index..
      child1[index]=0              #mutate index bit
    else:
      child1[index]=1
  return child1



#this is our main alog part that will run the whlole genetic algo part..

def genetic_algo(population,generation,population_size,x_df,y_df):
    popul=selection(population,x_df,y_df)    #selected a population parameter(population,ith generation)
    new_population=[]
    for j in range(int(population_size/2)):   #run the loop iterate by increment of j =2
       parent1,parent2=select_parent(popul,x_df,y_df)   #select two parents  passes parameter
       child1,child2= crossover(parent1, parent2)   #and senf them for crossover..
       child1 = mutation(child1,mutation_rate)  #then do mutation  .
       child2 = mutation(child2, mutation_rate)
       new_population.append(child1)
       new_population.append(child2)    #append to the new generation
    population = new_population           #made this new generation as our population and go in the loop again..
    return population

In [5]:
def start_genatics(population_size,generation,x_df,y_df):
    population = init_population(population_size,x_df)   #initialize intial population by calling init_population
    fitness_score=[]
    for i in range(len(population)):
      score=fitness_function(population[i],x_df,y_df)
      fitness_score.append(score)                      #calculate fitness score for initial population
      sorted_indices = np.argsort(fitness_score)[::-1]  #sort in descending order
    t=0
    list_ans = []
    list_ans.append([population[sorted_indices[0]],fitness_function(population[sorted_indices[0]],x_df,y_df)])  #append initial population bestest
    while t<generation:
       update_pop=genetic_algo(population,generation,population_size,x_df,y_df)   #call to the main algo function.
       fitness_score=[]
       for i in range(len(update_pop)):
           score=fitness_function(update_pop[i],x_df,y_df)
           fitness_score.append(score)                        #again calculate fitness score
       sorted_indices = np.argsort(fitness_score)[::-1]  #sort in descending order
       list_ans.append([update_pop[sorted_indices[0]],fitness_function(update_pop[sorted_indices[0]],x_df,y_df)])   #store best one
       population=update_pop       #initialize pop to update population
       t=t+1                       #gentration increment
    return list_ans

In [6]:
def main(x_df,y_df):
  res=[]
  for k in range(len(population_size)):
     for m in range(len(generation_size)):
      genetic_ans = start_genatics(population_size[k],generation_size[m],x_df,y_df)
      genetic_ans = pd.DataFrame(genetic_ans)
      result=genetic_ans.sort_values(by=1,ascending=False)  #sort dataframe in descending order based on fitness score by selecting column 1
      # print(result.head(10))
      # result_df.head(10)
      selected_ft=result.iloc[0][0]     #get 0th row 1th chromosome(selected_features which gives heigest performans)
      # print("total Features",len(cell_value))     #print number of feature
      count_OfSelceted_ft = selected_ft.count(1)               #calculate number of feature has been selected
      # print("selected features=",count_OfSelceted_ft )         #print selected feature
      accuracyScore=fitness_function(selected_ft,x_df,y_df)     #calculate best chromosome score
      # print("accuracy=",accuracyScore)
      res.append([population_size[k],generation_size[m],selected_ft,x_df.shape[1],count_OfSelceted_ft,accuracyScore])
  return res

**Global Search**

In [ ]:
df1=pd.read_csv(r'uploaded_file.csv')

# df1=df1.drop('ID',axis=1)
# print(df1.shape[1])

# from sklearn import preprocessing
# en=preprocessing.LabelEncoder()            #by LabelProcessing
# df1['class']=en.fit_transform(df1['class'])

x1_df = df1.iloc[:, 0:-1]
y1_df= df1.drop(df1.iloc[:, 0:-1], axis=1)
res= main(x1_df,y1_df)
res= pd.DataFrame(res)
res.columns =['Population', 'Generation', 'selected_features','total_features','total_selected_faetues','Accuracy']
# res.to_csv("/content/GlobalResult.csv")

**Sort Result in Descending Order**

In [ ]:
result=res.sort_values(by='Accuracy',ascending=False)  #sort dataframe in descending order based on fitness score by selecting column 1

**Select feature With heigest Acuuracy**

In [ ]:
s1_ft=result.iloc[0][2]     #get 0th row 1th habitate(selected_features which gives heigest performans)
# print("selected features is:",s1_ft)
# print("Total number of features ",result.iloc[0][3] )
# print("total selected Features",result.iloc[0][4])     #print number of feature
best_score=result.iloc[0][5]    #calculate best habitate vscore
# print("accuracy=",best_score)

**This final function is used for Local Search**

In [ ]:
def final(best_score,s_ft,max_sc,n1,xxdf,yydf,ft_dataset):
 numft=len(s_ft)
 if(best_score>=max_sc):
  minimize2_df=select_feature(s_ft,xxdf)
  # dataset="/content/With_"+str(numft)+"_Feature_Dataset.csv"   #for create selected feature dataset for each itration
  max_sc=best_score
  ans=main(minimize2_df,yydf)
  ans= pd.DataFrame(ans)
  ans.columns =['Population', 'Generation', 'selected_features','total_features','total_selected_faetues','Accuracy']
  # st="/content/Local_Search_Iteration5_result_"+str(n1)+".csv"     #for create result csv for each itration
  # ans.to_csv(st)
  n1=int(n1)
  n1=n1+1
  result=ans.sort_values(by='Accuracy',ascending=False)  #sort dataframe in descending order based on fitness score by selecting column 1
  select_ft=result.iloc[0][2]     #get 0th row 1th habitate(selected_features which gives heigest performans)
  best_sc=result.iloc[0][5]    #calculate best habitate vscore
  minimize2_df = pd.concat([minimize2_df,y1_df], axis=1)
  # minimize2_df.to_csv(dataset,index=False)        #for generate dataset
  ft_dataset=minimize2_df
  y2_df= minimize2_df.drop(minimize2_df.iloc[:, 0:-1], axis=1)
  final(best_sc,select_ft,max_sc,n1,minimize2_df,yydf,ft_dataset)
 else:
#   dataset="/content/With_"+str(numft)+"_Feature_Dataset.csv"   #for create selected feature dataset for each itration
  ft_dataset.to_csv('result.csv',index=False)        #for generate dataset
  # print("Search is Ended With best Result with "+str(n1)+" Itration")

**Call Final Function for Local Search**

In [ ]:
n=1              #set n=1 to itration How many local search Done
max_score=0       #intialize max_score
final_dataset=pd.DataFrame()
final(best_score,s1_ft,max_score,n,x1_df,y1_df,final_dataset)